In [1]:
#!/usr/bin/env python
import sys
import os
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
from firebase_admin import db
import serial
from picamera import PiCamera
from time import sleep
from pyfcm import FCMNotification
from firebase import firebase
import json


camera = PiCamera()
cred = credentials.Certificate('/home/pi/notebooks/firebase.json')
firebase_admin.initialize_app(cred, {'storageBucket': 'um170915.appspot.com'})
DETECTION_BOUNDARY = 45.0
DETECTION_LEVEL = 10 #(10/2) seconds
FIREBASE_URL = 'https://um170915.firebaseio.com/'
FIREBASE_API_KEY = "AAAACP7yGag:APA91bH9kExkuFoEs57P-s3NTfqkopCZVUdFKHwDutl7QV-VyqOvSv_LJQBwJiwnO0QN9Rey2-YfOXm5rUaHGfUumki5e99cBsNMjldIpD2tzZAtkHNfg7jQa3ttfYEnT2d5PCivef7Y"
fire = firebase.FirebaseApplication(FIREBASE_URL, None)



def sendMessage(pulse, sound):
    push_service = FCMNotification(api_key = FIREBASE_API_KEY)
    userTokens = fire.get('/Tokens', None)
    registration_ids = []
    for key, value in userTokens.items():
        registration_ids.append(key)
    return push_service.notify_multiple_devices(registration_ids=registration_ids, 
                                                message_title = "Baby Danger", message_body= "status")
def sendData(pulse):
    s = str(pulse)
    result = fire.post("/users", {'Heart': s})
    
if __name__ == "__main__":
    ser = serial.Serial('/dev/ttyUSB0', 9600)
    ser.flushInput()
    first = True
    while True:
        try:
            lineBytes = ser.readline()
            line = lineBytes.decode('utf-8')
            data = line.split(":")
            if (len(data) == 2):
                pulse = int(data[0])
                sound = int(data[1])       
                print("Input pulse {} , sound {}".format(pulse, sound))
                sendData(pulse)
                if((sound > 200 or pulse > 140 or pulse < 80) and first):
                    camera.start_preview()
                    camera.capture('/home/pi/image.png')
                    camera.stop_preview()                
                    bucket = storage.bucket()
                    blob = bucket.blob(os.path.basename('/home/pi/image.png'))
                    blob.upload_from_filename('/home/pi/image.png')
                    sendMessage(pulse, sound)
                    first = False
        except KeyboardInterrupt:
            break

    print("Server terminated")

ValueError: could not convert string to float: '\x00\x00201'